# AI Foundations: Working with LLMs and Tools

This notebook demonstrates fundamental concepts and techniques for working with Large Language Models (LLMs), particularly focusing on the OpenAI API (other LLM providers have a similar interface). The notebook is divided into three main sections that showcase different aspects of LLM interactions.

## 1. The Basics - Making an API Request
Learn the fundamentals of making API requests to OpenAI's services:
- Setting up the OpenAI client and environment
- Structuring JSON output formats
- Making basic chat completion requests with GPT-4o (or GPT-4o-mini)
- Understanding API response structures

## 2. Demystifying LLM Tools
Explore how to enhance LLM capabilities with custom tools:
- Implementation of a dummy stock price tool
- Creating and managing tool definitions
- Handling tool calls and responses
- Building conversational flows with tool integration

## 3. Advanced Prompting Techniques
Outline a few key prompting strategies:
- Chain of Thought (CoT) prompting
- One-shot / few-shot prompting examples
- Structured financial analysis prompts
- Response parsing and formatting

### Required Packages
The notebook uses several key Python packages:
- **openai**: For interacting with OpenAI's API
- **python-dotenv** and **getpass**: For environment variable management

# 1. The basics - making an API request

This section demonstrates how to make basic API requests to OpenAI's services using a structured JSON format. We'll create a financial analysis system that:

- Sets up the OpenAI client with an API key
- Defines a structured JSON template for financial analysis
- Makes a chat completion request with specific parameters:
  - `model`: Using GPT-4o (or GPT-4o-mini)
  - `temperature`: Lower temperature for more deterministic outputs
  - `max_tokens`: Limited to 1024 for response length
  - `seed`: Set to 42 for reproducibility
  - `response_format`: Enforcing JSON output

The example analyzes NVIDIA stock, returning detailed financial metrics, analyst opinions, trends, and key risks in a structured JSON format. This demonstrates how to:
1. Structure complex prompts with system and user messages
2. Enforce specific response formats
3. Handle and parse API responses

In [ ]:
# upgrade pip to latest version
!pip3 install --upgrade pip

In [ ]:
# install notebook package dependencies
!pip3 install openai python-dotenv

In [ ]:
# import required packages
from openai import OpenAI
import json
from dotenv import load_dotenv
from pprint import pprint
import re
import os
import getpass

# load env file
load_dotenv()

In [2]:
# or set openai api key
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [35]:
# initialize the OpenAI client
client = OpenAI()

In [ ]:
# Define the expected JSON structure for the financial analysis output
# This template specifies all the fields that should be included in the response
# including company metrics, analyst opinions, financial data, trends and risks

json_output_structure = """
{
  "ticker": "",
  "company_name": "",
  "current_price": 0,
  "price_change_24h": 0,
  "market_cap": 0,
  "pe_ratio": 0,
  "eps": 0,
  "dividend_yield": 0,
  "52_week_high": 0,
  "52_week_low": 0,
  "recommendation": "",
  "analyst_opinions": [
    {
      "analyst": "",
      "rating": "",
      "target_price": 0
    }
  ],
  "financial_summary": {
    "revenue": 0,
    "net_income": 0,
    "total_assets": 0,
    "total_liabilities": 0
  },
  "trends": [
    {
      "date": "",
      "event": "",
      "impact": ""
    }
  ],
  "key_risks": [
    {
      "risk": "",
      "potential_impact": ""
    }
  ]
}
"""

# Create a chat completion request to analyze NVIDIA stock

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are an expert financial analyst that specializes in analyzing technology companies. \
      You respond in a JSON format, and the response should include the following keys: " + json_output_structure},
    {"role": "user", "content": "Provide me with an analysis of NVIDIA"}
  ],
  temperature=0.7,
  max_tokens=1024,
  seed=42,
  response_format={ "type": "json_object" },
  
)

print(completion.choices[0].message.content)


In [ ]:
#pretty print completion variables
pprint(vars(completion))

# 2. Demystifying LLM Tools

This section demonstrates how to enhance LLM capabilities by integrating external tools through LLM function calling. We'll build a simple stock price lookup system that:

- Implements a dummy stock price tool that returns prices for specific companies
- Defines tool specifications using OpenAI's function schema:
  - Tool name and description
  - Required parameters and their types
  - Expected response format

The example showcases a complete conversation flow that:
1. Accepts user input about stock prices
2. Determines when to call the tool
3. Executes the appropriate function
4. Returns a natural language response incorporating the tool's data

In [37]:
# define dummy tool function
def get_current_stock_price(company):
    if 'nvidia' in company.lower():
        return json.dumps({'company': 'NVIDIA', 'stock_price': '135.37'})
    elif 'amd' in company.lower():
        return json.dumps({'company': 'AMD', 'stock_price': '153.44'})
    elif 'intel' in company.lower():
        return json.dumps({'company': 'Intel', 'stock_price': '23.20'})
    else:
        return json.dumps({"company": company, "stock_price": "unknown"})


In [38]:
# function to run the conversation
def run_conversation(user_input):
    # capture user input
    messages = [{"role": "user", "content": user_input}]
    # define the tools
    tools = [
        {
            'type': 'function',
            'function': {
                'name': 'get_current_stock_price',
                'description': 'Get current stock price information.',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'company': {
                            'type': 'string',
                            'description': 'the company name, e.g., NVIDIA'
                        }
                        },
                        'required': ['company'],
                    }
                }
            }
    ]

    # call model and give it tools
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        tools=tools,
        tool_choice='auto',
    )

    # print out model response
    response_message = response.choices[0].message
    print('Model Response:')
    pprint(vars(response_message))
    print()

    # print out if model chose to call a tool
    tool_calls = response_message.tool_calls
    if tool_calls:
        print('Tool Calls:')
        pprint(vars(tool_calls[0]))
        print()
    
    # if a tool was called, extract tool arguments and execute tool
    if tool_calls:
        available_functions = {
            "get_current_stock_price": get_current_stock_price,
        }
        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                company=function_args.get("company"),
            )
            # append tool response to messages
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            })
        # call model again with updated messages
        second_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        return second_response.choices[0].message.content

In [ ]:
# Test function calling
user_input = input("What can I help you with: ")
response = run_conversation(user_input)

print(("Tool Output:"))
print(f'User: {user_input}')
print(f'Assistant: {response}')

# 3. Advanced Prompting Techniques

This section explores more advanced prompting strategies to improve LLM responses. We'll demonstrate:

Chain of Thought (CoT) Prompting:
- Guides the model through step-by-step reasoning
- Uses explicit thinking markers (`<thinking>` vs vs `<output>` vs `<message>`)
- Improves response consistency and reliability

Few-Shot Learning:
- Provides example interactions for the model to learn from
- Includes sample financial analyses for NVIDIA and Intel
- Demonstrates proper response formatting and reasoning

Key Features:
- Response parsing to separate internal reasoning from external communication
- Interactive conversation loop for follow-up questions
- Pattern matching to extract different response components

In [40]:
# system prompt (chain of thought and few-shot prompt)
cot_financial_prompt = """
You are a financial analysis expert specializing in generating structured, JSON-based financial summaries for stocks. 
Think step by step to ensure consistency in your analysis for each stock, generate output in JSON format, and respond conversationally to user questions about the analysis.

Follow these examples:

Example 1:
User: Generate a financial analysis for NVIDIA (ticker: NVDA).

Assistant: <thinking> 
Step-by-Step Plan:
1. Identify the company and ticker symbol.
2. Retrieve and calculate key financial metrics like current price, market cap, P/E ratio, EPS, and dividend yield.
3. Gather and structure analyst opinions, financial summary, trends, and key risks.
4. Output the data in JSON format using the predefined keys.
</thinking>
<output> 
{
  "ticker": "NVDA",
  "company_name": "NVIDIA Corporation",
  "current_price": 500,
  "price_change_24h": 5.2,
  "market_cap": 1200000000000,
  "pe_ratio": 102.5,
  "eps": 4.89,
  "dividend_yield": 0.08,
  "52_week_high": 520,
  "52_week_low": 300,
  "recommendation": "Strong Buy",
  "analyst_opinions": [
    {
      "analyst": "Morgan Stanley",
      "rating": "Buy",
      "target_price": 550
    },
    {
      "analyst": "Goldman Sachs",
      "rating": "Hold",
      "target_price": 510
    }
  ],
  "financial_summary": {
    "revenue": 30000000000,
    "net_income": 6000000000,
    "total_assets": 45000000000,
    "total_liabilities": 18000000000
  },
  "trends": [
    {
      "date": "2024-11-01",
      "event": "Q3 earnings report",
      "impact": "Positive"
    }
  ],
  "key_risks": [
    {
      "risk": "Chip supply constraints",
      "potential_impact": "High"
    }
  ]
}
</output>

User: Why is NVIDIA rated as a "Strong Buy"?

Assistant: 
<thinking> 
Analyze the recommendation based on the JSON data, especially considering recent positive events, high market cap, strong EPS, and analyst ratings.
</thinking>
<message> 
NVIDIA has received a "Strong Buy" rating due to its strong financial performance, high market cap, and favorable analyst opinions. The recent Q3 earnings report showed positive growth, and major analysts like Morgan Stanley have set high target prices, indicating confidence in NVIDIA’s potential for future gains.
</message>

Example 2:
User: Generate a financial analysis for Intel (ticker: INTC).

Assistant: <thinking> 
Step-by-Step Plan:
1. Identify the company and ticker symbol.
2. Retrieve and calculate key financial metrics like current price, market cap, P/E ratio, EPS, and dividend yield.
3. Gather and structure analyst opinions, financial summary, trends, and key risks.
4. Output the data in JSON format using the predefined keys.
</thinking>
<output> 
{
  "ticker": "INTC",
  "company_name": "Intel Corporation",
  "current_price": 35,
  "price_change_24h": -0.8,
  "market_cap": 140000000000,
  "pe_ratio": 9.5,
  "eps": 3.7,
  "dividend_yield": 2.8,
  "52_week_high": 42,
  "52_week_low": 25,
  "recommendation": "Hold",
  "analyst_opinions": [
    {
      "analyst": "JP Morgan",
      "rating": "Sell",
      "target_price": 30
    },
    {
      "analyst": "Barclays",
      "rating": "Hold",
      "target_price": 37
    }
  ],
  "financial_summary": {
    "revenue": 80000000000,
    "net_income": 16000000000,
    "total_assets": 150000000000,
    "total_liabilities": 50000000000
  },
  "trends": [
    {
      "date": "2024-10-25",
      "event": "Dividend announcement",
      "impact": "Neutral"
    }
  ],
  "key_risks": [
    {
      "risk": "Increased competition in semiconductor industry",
      "potential_impact": "Medium"
    }
  ]
}
</output>

User: What are the main risks for Intel right now?

Assistant: 
<thinking> 
Identify key risks from the JSON data and explain them, particularly focusing on competition within the industry.
</thinking>
<message> 
One of the main risks for Intel is increased competition within the semiconductor industry. As more companies invest in advanced semiconductor technology, Intel faces pressure to innovate and maintain its market share. This competitive environment can impact Intel’s growth potential if it fails to keep up with industry advancements.
</message>

Use this format to generate financial summaries in <output></output> tags and respond to additional user questions in <message></message> tags.
"""


In [41]:
# get response from model chat completion
def get_response(messages):
  completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    temperature=1,
    max_tokens=1024,
    seed=42
    
  )
  return completion.choices[0].message.content

In [42]:
# parse response to extract thinking and message content
def parse_response(response):
    thinking_pattern = r"<thinking>(.*?)</thinking>"
    output_pattern = r"<output>(.*?)</output>"
    message_pattern = r"<message>(.*?)</message>"

    thinking = re.findall(thinking_pattern, response, re.DOTALL)
    output = re.findall(output_pattern, response, re.DOTALL)
    message = re.findall(message_pattern, response, re.DOTALL)

    return (thinking[0] if thinking else "", output[0] if output else "", message[0] if message else "")

In [ ]:
# initialize message history
messages=[
    {"role": "system", "content": cot_financial_prompt}
  ]

while True:
  # get user input
  user_input = input("Enter your message...")
  if user_input.lower() == "quit":
    break

  # append user input to message history
  messages.append({"role": "user", "content": user_input})
  print(f'User: {user_input}')

  # call model with message history
  response = get_response(messages)

  # append assistant output to message history
  messages.append({"role": "assistant", "content": response})

  # parse model response
  thinking, output, message = parse_response(response)
  print(f'Assistant thinking (internal): {thinking}')
  print(f'Assistant output (JSON): {output}')
  print(f'Assistant message (external): {message}')


### Practice Exercise 1

Build a new LLM pipeline that does comparative analysis between two stocks.

In [ ]:
# Step 1. Define a new system prompt

In [ ]:
# Step 2. Adjust the message parsing function

In [ ]:
# Step 3. Build the new LLM pipeline and test it with a comparative analysis between two stocks of your choice.